In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

In [3]:
df = pd.read_csv('Bib_skoky\David-50-pokus01\Xsens_DOT_4_D422CD006380_20250409_104913.csv', skiprows=7, sep=",")
df.head(10)

,PacketCounter,SampleTimeFine,Euler_X,Euler_Y,Euler_Z,Acc_X,Acc_Y,Acc_Z,Gyr_X,Gyr_Y,Gyr_Z
0,0,1607860018,-34.950737,-86.538986,66.922844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1607876685,-34.957100,-86.542999,66.933441,9.742603,-0.379648,0.488553,0.154169,-0.118958,-1.370459
2,2,1607893352,-34.948639,-86.544075,66.922592,9.765743,-0.408807,0.513513,0.623817,0.034199,-1.291495
3,3,1607910019,-34.951221,-86.542152,66.936539,9.757920,-0.406964,0.537707,0.626596,0.150212,-1.164430
4,4,1607926686,-34.931854,-86.538811,66.924667,9.783468,-0.403216,0.478358,0.522723,0.270464,-1.186777
5,5,1607943353,-34.895245,-86.535927,66.881142,9.762775,-0.393119,0.498226,-0.595982,0.337018,-1.301138
6,6,1607960020,-34.847225,-86.534798,66.826706,9.723853,-0.433274,0.508861,-0.349825,0.254244,-1.389585
7,7,1607976687,-34.833431,-86.538918,66.803223,9.739558,-0.414420,0.533203,-0.533762,-0.057225,-1.467601
8,8,1607993354,-34.845669,-86.547760,66.829964,9.714067,-0.409650,0.567066,0.767376,-0.390056,-1.488687
9,9,1608010021,-34.903389,-86.556335,66.904160,9.745429,-0.356492,0.540494,0.857942,-0.476601,-1.340786


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PacketCounter   672 non-null    int64  
 1   SampleTimeFine  672 non-null    int64  
 2   Euler_X         672 non-null    float64
 3   Euler_Y         672 non-null    float64
 4   Euler_Z         672 non-null    float64
 5   Acc_X           672 non-null    float64
 6   Acc_Y           672 non-null    float64
 7   Acc_Z           672 non-null    float64
 8   Gyr_X           672 non-null    float64
 9   Gyr_Y           672 non-null    float64
 10  Gyr_Z           672 non-null    float64
dtypes: float64(9), int64(2)
memory usage: 57.9 KB


In [5]:
df.describe()

,PacketCounter,SampleTimeFine,Euler_X,Euler_Y,Euler_Z,Acc_X,Acc_Y,Acc_Z,Gyr_X,Gyr_Y,Gyr_Z
count,672.000000,6.720000e+02,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000
mean,335.500000,1.613452e+09,-4.921456,-84.847278,27.905753,9.550452,-0.085446,0.219769,-0.525057,0.137778,-1.235772
std,194.133974,3.235631e+06,54.363654,4.980459,56.906307,6.681907,3.272399,6.786553,52.971886,66.506294,12.670521
min,0.000000,1.607860e+09,-179.454910,-89.230774,-170.980927,-30.049109,-36.249966,-95.211655,-446.554688,-631.761108,-117.451035
25%,167.750000,1.610656e+09,-32.229729,-86.782473,24.049617,9.719180,-0.409477,0.448186,-1.069538,-0.702206,-1.682524
50%,335.500000,1.613452e+09,-10.645339,-86.464958,33.125731,9.748317,-0.337362,0.521022,0.051379,0.067003,-1.280193
75%,503.250000,1.616248e+09,-0.113350,-86.098917,64.197803,9.766688,0.123208,0.580052,0.598860,0.772855,-0.936746
max,671.000000,1.619044e+09,179.643509,-59.658253,162.688309,96.797470,40.732857,87.241074,423.890625,542.825806,63.425293


In [6]:
# I zero the clock at the first row, this makes all subsequent rows relative to test start
df['real_ticks'] = df['SampleTimeFine'] - df['SampleTimeFine'].iloc[0] 

# Converting the ticks to seconds and milliseconds
df['time_s'] = df['real_ticks'] * 1e-6

# Convert seconds to milliseconds
df['time_ms'] = df['time_s'] * 1000
#  show the actual intervals between samples
df['delta_ms'] = df['time_ms'].diff()

In [8]:
df[['time_ms', 'time_s', 'delta_ms']].describe()

,time_ms,time_s,delta_ms
count,672.000000,672.000000,6.710000e+02
mean,5591.778500,5.591778,1.666700e+01
std,3235.630951,3.235631,5.193370e-13
min,0.000000,0.000000,1.666700e+01
25%,2795.889250,2.795889,1.666700e+01
50%,5591.778500,5.591778,1.666700e+01
75%,8387.667750,8.387668,1.666700e+01
max,11183.557000,11.183557,1.666700e+01


In [10]:
# Frequency Calculation:
# Find out how fast they sampled
mean_delta_ms = df['delta_ms'].mean()
fs = 1000.0 / mean_delta_ms
print(f"Sampling frequency: {fs} Hz")

Sampling frequency: 59.99880002399953 Hz


In [29]:
print('Z axis acceleration statistics of right foot IMU vs Time in Seconds, Milliseconds and Delta in milliseconds:\n')
df[['time_ms', 'time_s', 'delta_ms', 'Acc_Z']].describe()

Z axis acceleration statistics of right foot IMU vs Time in Seconds, Milliseconds and Delta in milliseconds:



,time_ms,time_s,delta_ms,Acc_Z
count,672.000000,672.000000,6.710000e+02,672.000000
mean,5591.778500,5.591778,1.666700e+01,0.219769
std,3235.630951,3.235631,5.193370e-13,6.786553
min,0.000000,0.000000,1.666700e+01,-95.211655
25%,2795.889250,2.795889,1.666700e+01,0.448186
50%,5591.778500,5.591778,1.666700e+01,0.521022
75%,8387.667750,8.387668,1.666700e+01,0.580052
max,11183.557000,11.183557,1.666700e+01,87.241074


In [7]:
acc_magnitude = np.sqrt(df["Acc_X"]**2 + df["Acc_Y"]**2 + df["Acc_Z"]**2)

acc_magnitude.describe()

count    672.000000
mean      10.799969
std        8.717990
min        0.000000
25%        9.748042
50%        9.771145
75%        9.794334
max      106.665667
dtype: float64